Define the webcam



In [1]:
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

In [4]:
def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)

  # get photo data
  data = eval_js('takePhoto({})'.format(quality))
  # get OpenCV format image
  img = js_to_image(data) 
  
  # call our darknet helper on webcam image
  detections, width_ratio, height_ratio = darknet_helper(img, width, height)

  # loop through detections and draw them on webcam image
  for label, confidence, bbox in detections:
    left, top, right, bottom = bbox2points(bbox)
    left, top, right, bottom = int(left * width_ratio), int(top * height_ratio), int(right * width_ratio), int(bottom * height_ratio)
    cv2.rectangle(img, (left, top), (right, bottom), class_colors[label], 2)
    cv2.putText(img, "{} [{:.2f}]".format(label, float(confidence)),
                      (left, top - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                      class_colors[label], 2)
  # save image
  cv2.imwrite(filename, img)

  return filename

In [ ]:
'''try:
  filename = take_photo('photo.jpg')
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))'''

In [5]:
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [ ]:
'''# start streaming video from webcam for testing
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0 
while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    # convert JS response to OpenCV Image
    frame = js_to_image(js_reply["img"])

    # create transparent overlay for bounding box
    bbox_array = np.zeros([480,640,4], dtype=np.uint8)'''

Clone and build yolov5

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt  # install dependencies

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Test image

In [ ]:
!python detect.py --weights yolov5s.pt --img 640 --conf 0.25 --source data/images/
Image(filename='runs/detect/exp/zidane.jpg', width=600)

Download a video for testing

In [ ]:
#!wget "https://drive.google.com/uc?export=download&id=1hFDIiJbG_lnJq-hEMWNUb4dVPNUOQwZc" -O "test.mp4"
#https://drive.google.com/file/d/1j3LaWiI2z0pIYCqJg_2SLXMrqpGZBGTr/view?usp=sharing
!wget "https://drive.google.com/uc?export=download&id=1j3LaWiI2z0pIYCqJg_2SLXMrqpGZBGTr" -O "test.mp4"

Test the video

In [ ]:
!python3 detect.py --weights yolov5s.pt --source test.mp4 --conf 0.25 --name test_result.mp4

In [ ]:
!pip install ffmpeg

Compress the result of the testing for displaying. Google Colab could not handle a large file

In [1]:
# compress the video result
!ffmpeg -i runs/detect/test_result.mp4/test.mp4 -vcodec h264 -b:v 1000k -acodec mp2 output.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

Display the result of video testing

In [ ]:
#Display the video result after compressed

import time
import glob
import torch
import os

import argparse
from sys import platform

from IPython.display import HTML
from base64 import b64encode

# compress video
path_video = 'output.mp4'

# Show video
mp4 = open(path_video,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=800 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

Using webcam as the input to run the multiple object detection program. It is not recommeded because it is running very slow


In [ ]:
# multiple object detection with webcam using pretrained model (Not recommeded)
'''
import torch
from torchvision import transforms
import torch.backends.cudnn as cudnn
from numpy import random

# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
bbox = ''
# Load the model and set the hyperparameters up
#model = torch.hub.load('ultralytics/yolov5', 'custom', path_or_model='yolov5s.pt')
#model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model = torch.hub.load('ultralytics/yolov5', 'custom', path_or_model='yolov5s.pt', force_reload=True, verbose = True)
cudnn.benchmark = True
#model.verbose = False
#model.autoshape()
model.conf = 0.25  # confidence threshold (0-1)
model.iou = 0.45  # NMS IoU threshold (0-1)
model.classes = None #[0, 15, 16]  # (optional list) filter by class, i.e. = [0, 15, 16] for persons, cats and dogs

while True:
  js_reply = video_frame(label_html, bbox)
  if not js_reply:
    break
  
  # convert JS response to OpenCV Image
  frame = js_to_image(js_reply["img"])
  #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
  #frame = cv2.flip(frame, 1)
  # create transparent overlay for bounding box
  bbox_array = np.zeros([480,640,4], dtype=np.uint8) 
  
  result = model(frame)
  
  Class_name = result.pandas().xyxy[0].name
  Confidence = result.pandas().xyxy[0].confidence
  colors = [[random.randint(0, 255) for _ in range(3)] for _ in range(len(Class_name))]
  Xmin = result.pandas().xyxy[0].xmin
  Ymin = result.pandas().xyxy[0].ymin
  Xmax = result.pandas().xyxy[0].xmax
  Ymax = result.pandas().xyxy[0].ymax

  #print(result.pandas().xyxy[0])
  
  _, _, h, w = result.s
  img_height, img_width, _ = frame.shape
  
  width_ratio = img_width/w
  height_ratio = img_height/h

  #names = model.module.names if hasattr(model, 'module') else model.names
  #colors = [[random.randint(0, 255) for _ in range(3)] for _ in range(len(names))] 

  for i in range(0, len(Class_name)):
    xmin = int(Xmin[i] + 0.5)
    ymin = int(Ymin[i] + 0.5)
    xmax = int(Xmax[i] + 0.5)
    ymax = int(Ymax[i] + 0.5)

    xmin = int(xmin * width_ratio)
    ymin = int(ymin * height_ratio)
    xmax = int(xmax * width_ratio)
    ymax = int(ymax * height_ratio)

    bbox_array = cv2.rectangle(bbox_array, (xmin, ymin), (xmax, ymax), colors[i], 2)
    bbox_array = cv2.putText(bbox_array, Class_name[i],
                        (xmin, ymin - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                        colors[i], 2)

  bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
  
  # convert overlay of bbox into bytes
  bbox_bytes = bbox_to_bytes(bbox_array)
  
  # update bbox so next frame gets new overlay
  bbox = bbox_bytes'''

USE THIS ONE TO RUN THE PROGRAM

In [ ]:
 # Real multiple object detection with webcam using pretrained model with multi-threading
'''
The labelling is still not accurate, caused by canvas for labelling is shifted. Need to be fixed in javascript part.

The hyperparameters are not optimal yet. Need to be tuned.

This part is using multithreading in order run the detection in real-time  
'''

import torch
from torchvision import transforms
import torch.backends.cudnn as cudnn
from numpy import random
import threading
import queue

# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
bbox = ''
# Load the model and set the hyperparameters up
model = torch.hub.load('ultralytics/yolov5', 'custom', path_or_model='yolov5s.pt', force_reload=True, verbose = True)
cudnn.benchmark = True
model.conf = 0.25  # confidence threshold (0-1)
model.iou = 0.45  # NMS IoU threshold (0-1)
model.classes = None #[0, 15, 16]  # (optional list) filter by class, i.e. = [0, 15, 16] for persons, cats and dogs

my_queue = queue.Queue()

def storeInQueue(f):
  def wrapper(*args):
    my_queue.put(f(*args))
  return wrapper

@storeInQueue
def detection(frame, bbox_array):
  result = model(frame)
  
  Class_name = result.pandas().xyxy[0].name
  Confidence = result.pandas().xyxy[0].confidence
  colors = [[random.randint(0, 255) for _ in range(3)] for _ in range(len(Class_name))]
  Xmin = result.pandas().xyxy[0].xmin
  Ymin = result.pandas().xyxy[0].ymin
  Xmax = result.pandas().xyxy[0].xmax
  Ymax = result.pandas().xyxy[0].ymax
  
  _, _, h, w = result.s
  img_height, img_width, _ = frame.shape
  
  width_ratio = img_width/w
  height_ratio = img_height/h

  for i in range(0, len(Class_name)):
    xmin = int(Xmin[i] + 0.5)
    ymin = int(Ymin[i] + 0.5)
    xmax = int(Xmax[i] + 0.5)
    ymax = int(Ymax[i] + 0.5)

    xmin = int(xmin * width_ratio)
    ymin = int(ymin * height_ratio)
    xmax = int(xmax * width_ratio)
    ymax = int(ymax * height_ratio)

    bbox_array = cv2.rectangle(bbox_array, (xmin, ymin), (xmax, ymax), colors[i], 2)
    bbox_array = cv2.putText(bbox_array, Class_name[i],
                        (xmin, ymin - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                        colors[i], 2)
  bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
  # convert overlay of bbox into bytes
  bbox_bytes = bbox_to_bytes(bbox_array)
  
  # update bbox so next frame gets new overlay
  # bbox = bbox_bytes
  return bbox_bytes

while True:
  js_reply = video_frame(label_html, bbox)
  if not js_reply:
    break
  
  # convert JS response to OpenCV Image
  frame = js_to_image(js_reply["img"])

  # create transparent overlay for bounding box
  bbox_array = np.zeros([480,640,4], dtype=np.uint8)

  t1 = threading.Thread(target=detection, args=(frame, bbox_array))
  t1.start()
  #t1.join()
  my_data = my_queue.get()
  bbox = my_data


OPTIONAL

In [ ]:
# Download COCO val2017
torch.hub.download_url_to_file('https://github.com/ultralytics/yolov5/releases/download/v1.0/coco2017val.zip', 'tmp.zip')
!unzip -q tmp.zip -d ../ && rm tmp.zip

In [ ]:
# Run YOLOv5x on COCO val2017
!python test.py --weights yolov5x.pt --data coco.yaml --img 640 --iou 0.65

In [ ]:
# Download COCO test-dev2017
torch.hub.download_url_to_file('https://github.com/ultralytics/yolov5/releases/download/v1.0/coco2017labels.zip', 'tmp.zip')
!unzip -q tmp.zip -d ../ && rm tmp.zip # unzip labels
!f="test2017.zip" && curl http://images.cocodataset.org/zips/$f -o $f && unzip -q $f && rm $f  # 7GB,  41k images
%mv ./test2017 ../coco/images  # move to /coco

In [ ]:
# Run YOLOv5s on COCO test-dev2017 using --task test
!python test.py --weights yolov5s.pt --data coco.yaml --task test

In [ ]:
# Download COCO128
torch.hub.download_url_to_file('https://github.com/ultralytics/yolov5/releases/download/v1.0/coco128.zip', 'tmp.zip')
!unzip -q tmp.zip -d ../ && rm tmp.zip

In [ ]:
# Tensorboard (optional)
%load_ext tensorboard
%tensorboard --logdir runs/train

In [ ]:
# Weights & Biases (optional)
%pip install -q wandb  
!wandb login  # use 'wandb disabled' or 'wandb enabled' to disable or enable

In [ ]:
# Train YOLOv5s on COCO128 for 3 epochs
!python train.py --img 640 --batch 16 --epochs 3 --data coco128.yaml --weights yolov5s.pt --nosave --cache

In [ ]:
Image(filename='runs/train/exp/train_batch0.jpg', width=800)  # train batch 0 mosaics and labels
Image(filename='runs/train/exp/test_batch0_labels.jpg', width=800)  # test batch 0 labels
Image(filename='runs/train/exp/test_batch0_pred.jpg', width=800)  # test batch 0 predictions

In [ ]:
from utils.plots import plot_results 
plot_results(save_dir='runs/train/exp')  # plot all results*.txt as results.png
Image(filename='runs/train/exp/results.png', width=800)

In [ ]:
# Re-clone repo
%cd ..
%rm -rf yolov5 && git clone https://github.com/ultralytics/yolov5
%cd yolov5

In [ ]:
# Reproduce
for x in 'yolov5s', 'yolov5m', 'yolov5l', 'yolov5x':
  !python test.py --weights {x}.pt --data coco.yaml --img 640 --conf 0.25 --iou 0.45  # speed
  !python test.py --weights {x}.pt --data coco.yaml --img 640 --conf 0.001 --iou 0.65  # mAP

In [ ]:
# PyTorch Hub
import torch

# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Images
dir = 'https://github.com/ultralytics/yolov5/raw/master/data/images/'
imgs = [dir + f for f in ('zidane.jpg', 'bus.jpg')]  # batch of images

# Inference
results = model(imgs)
#results.print()  # or .show(), .save()
results.show()